In [23]:
import numpy as np
import pandas as pd
import time

# Define benchmark functions
def sphere(x):
    return np.sum(x**2)

def rosenbrock(x):
    return sum(100 * (x[i+1] - x[i]**2)**2 + (1 - x[i])**2 for i in range(len(x)-1))

def step_2(x):
    return np.sum(np.floor(x + 0.5)**2)

# Add more functions following the pattern above...
def quartic(x):
    return np.sum(7 * x**4 - 10 * x**3 + 5 * x**2)

def schwefel_2_21(x):
    return np.sum(abs(x) + np.prod(abs(np.sin(x))))

def schwefel_2_22(x):
    return -np.sum(np.sin(np.sqrt(abs(x))))


def foxholes(x):
    return np.sum(
      np.exp(-0.2 * np.sqrt(np.mean(x ** 2, axis=0))) + np.exp(np.sin(np.sqrt(np.mean(x ** 2, axis=0)))) + 0.1 *
    x.shape[0])

def kowalik(x):
    return np.sum([i * (x[j] - 1)**2 + (1 - x[j])**2 for j in range(len(x)) for i in range(1, 6)])

# Six-hump camel back Function
def six_hump_camel_back(x):
    return 4 * x[0]**2 - 2.1 * x[0]**4 + 1/3 * x[0]**6 + x[0] * x[1] - 4 * x[1]**2 + 4 * x[1]**4


# # Hartman 6 Function
# def hartman_6(x):
#     return -np.sum(np.array([0.3979, 0.4899, 0.6759, 0.7699, 0.9149, 1.0472]) * np.exp(-np.sum(np.array([1, 10, 100, 1000, 10000, 100000]) * ((x - np.array([0.1312, 0.2329, 0.5358, 0.8775, 0.9991, 0.7743]))**2), axis=1)))

# Levi Function N.13
def levi_n13(x):
    return np.sum(np.sin(np.sqrt(abs(x**2 + (1 + np.sin(10 * np.pi * x))**2))))

# Rastrigin Function
def rastrigin(x):
    return np.sum(x**2 - 10 * np.cos(2 * np.pi * x) + 10)

# Griewank Function
def griewank(x):
    return np.sum(x**2 / 4000 - np.cos(x) + 1)

# Ackley 1 Function
def ackley_1(x):
    return -20 * np.exp(-0.2 * np.sqrt(np.mean(x**2))) - np.exp(np.mean(np.cos(2 * np.pi * x))) + 20 + np.e


# def hartman_3(x):
#     x = np.atleast_2d(x)  # Ensure x is a 2D array for proper broadcasting
#     weights = np.array([0.3689, 0.4699, 1.0472, 1.5701, 0.7473])[:x.shape[1]]
#     return -np.sum(weights * np.exp(-np.sum(np.array([1, 10, 100, 1000]) * (x.T - np.array([0.1312, 0.2329, 0.5358, 0.8775]))**2, axis=1)))
def schwefel_2_26(x):
    return np.sum(np.abs(x) + np.sin(x**2))
def branin_rcos(x):
    return (x[1] - 5.1 / (4 * np.pi**2) * x[0]**2 + 5 * x[0] / np.pi - 6)**2 + 10 * (1 - 1 / (8 * np.pi)) * np.cos(x[0]) + 10
def goldstein_price(x):
    return (1 + (x[0] + x[1] + 1)**2 * (19 - 14 * x[0] + 3 * x[0]**2 - 14 * x[1] + 6 * x[0] * x[1] + 3 * x[1]**2)) * \
           (30 + (2 * x[0] - 3 * x[1])**2 * (18 - 32 * x[0] + 12 * x[0]**2 + 48 * x[1] - 36 * x[0] * x[1] + 27 * x[1]**2))

def penalized_1(x):
    term1 = 10 * np.sin(np.pi * x[0])**2
    term2 = sum((x[i] - 1)**2 * (1 + 10 * np.sin(np.pi * x[i] + 1)**2) for i in range(1, len(x)))
    term3 = (x[-1] - 1)**2 * (1 + np.sin(2 * np.pi * x[-1])**2)
    return term1 + term2 + term3

# def shekel_1(x):
#     a = np.array([[4, 4, 4, 4],
#                   [1, 1, 1, 1],
#                   [8, 8, 8, 8],
#                   [6, 6, 6, 6],
#                   [3, 7, 3, 7]])
#
#     b = np.array([0.1, 0.2, 0.2, 0.4, 0.4])
#
#     x = np.atleast_2d(x)  # Ensure x is a 2D array for proper broadcasting
#     return -np.sum((1 / (np.sum((x[:, np.newaxis, :] - a)**2, axis=-1) + b)), axis=-1)
#


# def shekel_2(x):
#     a = np.array([[4, 4, 4, 4],
#                   [1, 1, 1, 1],
#                   [8, 8, 8, 8],
#                   [6, 6, 6, 6],
#                   [3, 7, 3, 7]])
#
#     b = np.array([0.1, 0.2, 0.2, 0.4, 0.4])
#
#     return -np.sum((1 / (np.sum((x - a)**2, axis=1) + b))**2)
#
# def shekel_3(x):
#     a = np.array([[4, 4, 4, 4],
#                   [1, 1, 1, 1],
#                   [8, 8, 8, 8],
#                   [6, 6, 6, 6],
#                   [3, 7, 3, 7]])
#
#     b = np.array([0.1, 0.2, 0.2, 0.4, 0.4])
#
#     return -np.sum(np.sqrt(1 / (np.sum((x - a)**2, axis=1) + b)))

def adaptive_inertia_weight(iteration, max_iter, w_max=0.9, w_min=0.4):
    return w_max - (iteration / max_iter) * (w_max - w_min)

# Check for convergence (optional, based on problem specifics)
def has_converged(previous_best, current_best, threshold=1e-6):
    return np.abs(previous_best - current_best) < threshold


# Modified PSO Implementation
def run_modified_pso(obj_func, func_name, n_particles=30, n_dimensions=2, max_iter=100, bounds=(-10, 10)):
    
    start_time = time.time()  # Timing start
    particles = np.random.uniform(bounds[0], bounds[1], (n_particles, n_dimensions))
    velocity = np.zeros((n_particles, n_dimensions))
    pbest = np.copy(particles)
    pbest_values = np.array([obj_func(x) for x in particles])
    gbest_value = np.min(pbest_values)
    gbest = particles[np.argmin(pbest_values)]
    
    v_max = (bounds[1] - bounds[0]) / 2  # Adjust velocity clamping based on bounds
    
    previous_best = float('inf')
    
    iteration_details = []

    for t in range(max_iter):
        w = adaptive_inertia_weight(t, max_iter)  # Example of modifying the inertia weight over time
        for i in range(n_particles):
            r1, r2 = np.random.rand(2)
            velocity[i] = w * velocity[i] + \
                          2.5 * r1 * (pbest[i] - particles[i]) + \
                          2.5 * r2 * (gbest - particles[i])
            velocity[i] = np.clip(velocity[i], -v_max, v_max)
            particles[i] += velocity[i]
            particles[i] = np.clip(particles[i], bounds[0], bounds[1])
            current_value = obj_func(particles[i])
            if current_value < pbest_values[i]:
                pbest[i] = particles[i]
                pbest_values[i] = current_value
                if current_value < gbest_value:
                    gbest = particles[i]
                    gbest_value = current_value
        iteration_details.append({'Iteration': t+1, 'Best Value': gbest_value})

    convergence_time = time.time() - start_time

    return func_name, iteration_details, convergence_time

# Objective Functions List
# Objective Functions List
functions = {
    'Sphere': sphere,
    'Rosenbrock': rosenbrock,
    'Step 2': step_2,
    'Quartic': quartic,
    'Schwefel 2.21': schwefel_2_21,
    'Schwefel 2.22': schwefel_2_22,
    'Foxholes': foxholes,
    'Kowalik': kowalik,
    'Six_hump_camel_back': six_hump_camel_back,
    'Levi N.13': levi_n13,
    'Rastrigin': rastrigin,
    'Griewank': griewank,
    'Ackley 1': ackley_1,
    'Schwefel 2.26': schwefel_2_26,
    'Branin Rcos': branin_rcos,
    'Goldstein Price': goldstein_price,
    'Penalized 1': penalized_1
}
# Collecting iteration details for each function
all_iterations_df = pd.DataFrame()

for name, func in functions.items():
    func_name, iteration_details, convergence_time = run_modified_pso(func, name, max_iter=30)  # Assuming 30 iterations for demonstration
    temp_df = pd.DataFrame(iteration_details)
    temp_df['Function'] = func_name
    temp_df['Convergence Time'] = convergence_time
    all_iterations_df = pd.concat([all_iterations_df, temp_df], ignore_index=True)

# Save to Excel
all_iterations_df.to_excel('modified_pso_iteration_details.xlsx', index=False)

print("Modified PSO iteration details for each function saved to 'modified_pso_iteration_details.xlsx'.")


Modified PSO iteration details for each function saved to 'modified_pso_iteration_details.xlsx'.
